# 네이버 블로그 데이터 수집 및 전처리 파이프라인

네이버 검색을 통해 일자별로 수집된 URL 목록을 이용해 실제 블로그의 제목과 본문 데이터를 수집하고, 분석에 적합한 형태로 가공하는 전체 과정을 담고 있습니다. 데이터의 정확성과 효율성을 높이기 위해 다음과 같은 체계적인 단계를 거쳐 진행되었습니다.

In [ ]:
import pandas as pd
import numpy as np 
import os
import requests
from bs4 import BeautifulSoup
import time
from tqdm.auto import tqdm

### 1. 날짜 폴더에 흩어져 있는 파일 병합(중복 제거 - 링크 기준)
- 목표 : 일자별 폴더에 등록되어 있는 '8개 키워드(카카오톡 업데이트, 카카오톡 개편, 카톡 업데이트, 카톡 개편, 네이트온, 라인, 카카오톡, 카톡)'로 검색한 결과 파일들을 URL 기준으로 하나의 데이터 프레임에 통합합니다.
- 핵심 : set() 자료구조를 활용하여, 여러 번 수집되었을 수 있는 중복 URL을 이 단계에서 미리 제거하여 데이터의 정합성을 확보하고, 이후의 크롤링 작업 효율을 높입니다.

In [ ]:
# --------------------------------------------------------------------------
# 1. Settings
# --------------------------------------------------------------------------
# 일자별로 수집된 검색 결과 파일 저장 경로 지정(load file)
BASE_PATH = 'E:/데일리프로젝트/251120_카카오톡 네이버기사,블로그/data'
# 결과 파일 저장 경로 지정
OUTPUT_PATH = 'E:/데일리프로젝트/251120_카카오톡 네이버기사,블로그/merged_data_blog'

# 수집된 파일명(예: '251005_naver_blogs_links_final1')에 키워드 대신 숫자를 입력했었기 때문에 실제 어떤 키워드를 의미하는지를 매칭시키기 
keyword_map = {
    1: '카카오톡_업데이트',
    2: '카카오톡_개편',
    3: '카톡_업데이트',
    4: '카톡_개편',
    5: '네이트온',
    6: '라인',
    7: '카카오톡',
    8: '카톡'
}

# --------------------------------------------------------------------------
# 2. 파일 경로 수집 및 키워드별 그룹화
# --------------------------------------------------------------------------
print("파일 검색 시작")

# 키워드 번호(1~8)별로 파일 경로를 저장할 딕셔너리 생성
file_groups = {num: [] for num in range(1, 9)}

# BASE_PATH 아래 날짜 폴더가 있기 때문에 내부 폴더 검색하게 반복문 적용
for dir_name in os.listdir(BASE_PATH):
    dir_path = os.path.join(BASE_PATH, dir_name)
    
    if os.path.isdir(dir_path):
        # 폴더 내 파일 체크
        for file_name in os.listdir(dir_path):
            # 'final'로 끝나고 '.csv' 확장자를 가진 파일만 대상
            if 'final' in file_name and file_name.endswith('.csv'):
                try:
                    # 파일명에서 숫자 부분(e.g., 'final1' -> 1) 추출
                    file_num = int(file_name.split('final')[1].split('.csv')[0])
                    if file_num in file_groups:
                        full_path = os.path.join(dir_path, file_name)
                        file_groups[file_num].append(full_path)
                except (ValueError, IndexError):
                    # 파일명 형식이 예상과 다를 경우 오류 메시지 출력 후 패스
                    print(f"파일명 형식이 달라 건너뜁니다: {file_name}")

print("파일 그룹화 완료")
print("-" * 30)

# --------------------------------------------------------------------------
# 3. 그룹별 파일 병합, 중복 제거 및 저장
# --------------------------------------------------------------------------
# 결과 파일을 저장할 폴더가 없으면 새로 생성
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)
    print(f"'{OUTPUT_PATH}' 폴더를 생성했습니다.")

# 그룹화된 파일들을 병합 작업하기
for num, file_list in file_groups.items():
    if not file_list:
        print(f"키워드 그룹 {num} ({keyword_map[num]})에 해당하는 파일이 없습니다.")
        continue

    # 각 그룹의 모든 CSV 파일을 DataFrame으로 읽어 리스트에 담고 합치기
    df_list = [pd.read_csv(file) for file in file_list]
    
    combined_df = pd.concat(df_list, ignore_index=True)
    
    # 'URL' 컬럼을 기준으로 중복된 행 제거(첫번째 파일만 남김)
    deduplicated_df = combined_df.drop_duplicates(subset=['URL'], keep='first')
    
    # 결과 파일을 CSV로 저장(인코딩 꼭 적기!!)
    output_filename = f"combined_{num}_{keyword_map[num]}.csv"
    output_filepath = os.path.join(OUTPUT_PATH, output_filename)
    deduplicated_df.to_csv(output_filepath, index=False, encoding='utf-8-sig')
    
    # 작업 결과 출력
    print(f"'{output_filename}' 파일 생성 완료!")
    print(f" - 통합 전 총 데이터 수: {len(combined_df)}개")
    print(f" - 중복 제거 후 총 데이터 수: {len(deduplicated_df)}개")
    print("-" * 30)

print("모든 작업이 성공적으로 완료되었습니다.")

### 2. 병합된 파일에서 중요 키워드를 포함하지 않는 블로그 제외하기
- 목표: 분석과 관련 없는 데이터를 제거하여 분석의 정확도를 높입니다.
- 핵심: 제목에 '설정한 핵심 키워드'를 포함하지 않는 포스트를 본문 수집 전에 필터링합니다.

In [ ]:
# --------------------------------------------------------------------------
# 1. Settings
# --------------------------------------------------------------------------
# 1단계 결과 폴더 연결
INPUT_PATH = 'E:/데일리프로젝트/251120_카카오톡 네이버기사,블로그/merged_data_blog'

# 결과 파일 저장 경로 지정
FILTERED_OUTPUT_PATH = 'E:/데일리프로젝트/251120_카카오톡 네이버기사,블로그/filtered_data_blog'

# '제목' 컬럼에 키워드가 없으면 제거
KEYWORDS_TO_CHECK = ["카카오톡", "업데이트", "개편", "카톡", "네이트온", "라인"]

# --------------------------------------------------------------------------
# 2. 폴더 생성 및 필터링 작업 시작
# --------------------------------------------------------------------------
# 결과 파일을 저장할 폴더가 없으면 새로 생성
if not os.path.exists(FILTERED_OUTPUT_PATH):
    os.makedirs(FILTERED_OUTPUT_PATH)
    print(f"'{FILTERED_OUTPUT_PATH}' 폴더를 생성했습니다.")

print("제목 기반 키워드 필터링을 시작합니다...")
print("-" * 30)

# --------------------------------------------------------------------------
# 3. 파일 처리 및 필터링
# --------------------------------------------------------------------------
# INPUT_PATH 폴더에 있는 모든 파일(csv 파일만 대상으로) 체크
for filename in os.listdir(INPUT_PATH):
    if filename.endswith('.csv'):
        file_path = os.path.join(INPUT_PATH, filename)
        
        try:
            df = pd.read_csv(file_path)
            
            original_rows = len(df)
            
            # '제목' 컬럼이 있는 지 체크
            if '제목' not in df.columns:
                print(f"'{filename}' 파일에 '제목' 컬럼이 없어 건너뜁니다.")
                continue
                
            # 키워드 리스트를 OR 조건으로 묶어 하나의 검색 패턴으로 생성
            search_pattern = '|'.join(KEYWORDS_TO_CHECK)
            
            # '제목' 컬럼에서 키워드 패턴을 포함하는 모든 행 추출
            # na=False 옵션은 '제목'이 비어있는(NaN) 경우 False로 처리하여 오류를 방지함
            filtered_df = df[df['제목'].str.contains(search_pattern, na=False)].copy()
            
            # 필터링 후 남은 데이터의 행 개수 저장
            filtered_rows = len(filtered_df)
            
            # 필터링된 결과를 새로운 파일로 저장
            output_filename = f"filtered_{filename}"
            output_filepath = os.path.join(FILTERED_OUTPUT_PATH, output_filename)
            filtered_df.to_csv(output_filepath, index=False, encoding='utf-8-sig')
            
            # 작업 결과 출력
            print(f"파일 처리 완료: {filename}")
            print(f" - 원본 데이터 수: {original_rows}개")
            print(f" - 필터링 후 데이터 수: {filtered_rows}개")
            print(f" - 결과 저장 파일: {output_filename}")
            print("-" * 30)

        except Exception as e:
            # 파일 처리 중 문제가 발생하면 오류 메시지 출력
            print(f"'{filename}' 파일 처리 중 오류 발생: {e}")
            print("-" * 30)

print("모든 필터링 작업이 성공적으로 완료되었습니다.")

### 3. 파일 내 링크로 이동해 해당 블로그의 본문 내용과 작성일을 크롤링하기(iframe 이슈 처리)
- 목표: 정제된 URL 목록을 바탕으로, 실제 분석의 대상이 될 텍스트 데이터를 수집합니다.
- 핵심: requests와 BeautifulSoup4 라이브러리를 사용하였으며, 네이버 블로그의 HTML 구조를 분석하여 정확한 본문 영역을 타겟팅했습니다. 또한, 서버 부하를 줄이고 IP 차단을 방지하기 위해 각 요청 사이에 time.sleep(1)을 적용하는 'Polite Scraping' 원칙을 준수했습니다.

In [ ]:
# --------------------------------------------------------------------------
# 1. Settings
# --------------------------------------------------------------------------
# 2단계 결과 폴더 연결
INPUT_PATH = 'E:/데일리프로젝트/251120_카카오톡 네이버기사,블로그/filtered_data_blog'

# 결과 파일 저장 경로 지정
CRAWLED_OUTPUT_PATH = 'E:/데일리프로젝트/251120_카카오톡 네이버기사,블로그/collected_main_text_blog'

# 중간 저장 주기
SAVE_INTERVAL = 50

# --------------------------------------------------------------------------
# 2. 크롤링 함수(날짜, 본문 동시 수집)
# --------------------------------------------------------------------------
def scrape_blog_content(url):
    # 네이버 블로그 URL을 받아 (작성일, 본문내용) 튜플을 반환
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                          'AppleWebKit/537.36 (KHTML, like Gecko) '
                          'Chrome/91.0.4472.124 Safari/537.36'
        }
        
        # 1. iframe 확인
        initial_response = requests.get(url, headers=headers, timeout=10)
        if initial_response.status_code != 200:
            return None, f"페이지 요청 실패 (1단계): {initial_response.status_code}"
            
        initial_soup = BeautifulSoup(initial_response.text, 'html.parser')
        main_frame = initial_soup.find('iframe', id='mainFrame')
        
        # iframe이 있는 경우 - 대부분의 네이버 블로그
        if main_frame:
            real_blog_url = "https://blog.naver.com" + main_frame['src']
            content_response = requests.get(real_blog_url, headers=headers, timeout=10)
            if content_response.status_code != 200:
                return None, f"페이지 요청 실패 (2단계): {content_response.status_code}"
            

            soup = BeautifulSoup(content_response.text, 'html.parser')
        else:
            # iframe이 없는 특이 케이스 - 모바일 뷰 등
            soup = initial_soup

        # ---------------------------------------
        # (1) 날짜 추출 로직
        # ---------------------------------------
        date_text = None
        # 최신 에디터
        date_elem = soup.find('span', class_='se_publishDate')
        # 구버전 에디터
        if not date_elem:
            date_elem = soup.find('span', class_='date')
        if not date_elem:
            date_elem = soup.find('p', class_='date')
            
        if date_elem:
            date_text = date_elem.get_text(strip=True)
        else:
            date_text = "날짜 미상"

        # ---------------------------------------
        # (2) 본문 추출 로직
        # ---------------------------------------
        content_text = ""
        main_container = soup.find('div', class_='se-main-container')
        
        if main_container:
            content_text = main_container.get_text(separator='\n', strip=True)
        else:
            post_view_area = soup.find('div', id='postViewArea')
            if post_view_area:
                content_text = post_view_area.get_text(separator='\n', strip=True)
            else:
                content_text = "" # 본문을 못 찾은 경우 빈 문자열

        # 날짜, 본문 빈환
        return date_text, content_text

    except requests.exceptions.RequestException as e:
        return None, f"크롤링 오류 발생: {e}"
        
# --------------------------------------------------------------------------
# 3. 폴더 생성 및 크롤링 작업 시작
# --------------------------------------------------------------------------
if not os.path.exists(CRAWLED_OUTPUT_PATH):
    os.makedirs(CRAWLED_OUTPUT_PATH)
    print(f"'{CRAWLED_OUTPUT_PATH}' 폴더를 생성했습니다.")

print("블로그 작성일 및 본문 크롤링을 시작합니다. (중간 저장 기능 활성화)")
print("-" * 50)

# --------------------------------------------------------------------------
# 4. 파일 순회 및 크롤링 실행 (작성일/본문 동시 저장)
# --------------------------------------------------------------------------
all_files = [f for f in os.listdir(INPUT_PATH) if f.endswith('.csv')]

for filename in all_files:
    print(f"[{filename}] 파일 작업을 시작합니다...")
    
    # 원본 파일 경로와 최종 저장될 파일(.pkl) 경로
    source_file_path = os.path.join(INPUT_PATH, filename)
    output_filename = f"crawled_{os.path.splitext(filename)[0]}.pkl"
    output_filepath = os.path.join(CRAWLED_OUTPUT_PATH, output_filename)

    # --- 이어하기 ---
    if os.path.exists(output_filepath):
        print(f" -> '{output_filename}' 파일이 존재합니다. 작업을 이어서 시작합니다.")
        df = pd.read_pickle(output_filepath)
    else:
        print(f" -> 새로운 작업을 시작합니다.")
        df = pd.read_csv(source_file_path)
    
    # '본문' 컬럼 초기화
    if '본문' not in df.columns:
        df['본문'] = np.nan
        
    # '작성일' 컬럼도 초기화
    if '작성일' not in df.columns:
        df['작성일'] = np.nan

    # 크롤링이 아직 되지 않은 행들만 대상 - 본문이 비어있는 경우
    to_crawl = df[df['본문'].isnull()]
    
    if to_crawl.empty:
        print(" -> 모든 URL의 수집이 이미 완료되었습니다.")
        print("-" * 50)
        continue

    print(f" -> 총 {len(df)}개 중 {len(to_crawl)}개의 URL에 대한 수집을 시작합니다.")
    
    # 진행률 표시
    for index, row in tqdm(to_crawl.iterrows(), total=len(to_crawl)):
        date_val, content_val = scrape_blog_content(row['URL'])
        
        df.loc[index, '작성일'] = date_val
        df.loc[index, '본문'] = content_val
        
        # 서버 부하 방지
        time.sleep(1)
        
        # --- 중간 저장 ---
        current_progress_index = to_crawl.index.get_loc(index)
        if (current_progress_index + 1) % SAVE_INTERVAL == 0:
            df.to_pickle(output_filepath)
            tqdm.write(f" -> 중간 저장 완료 ({current_progress_index + 1}/{len(to_crawl)} 진행)")

    # 모든 작업이 끝난 후 최종 저장
    df.to_pickle(output_filepath)
    print(f" -> 완료! 최종 결과가 '{output_filename}' 파일로 저장되었습니다.")
    print("-" * 50)

print("모든 크롤링 작업이 성공적으로 완료되었습니다.")        

### 4. 수집 완료 후 결과 확인
- 목표 : 수집된 결과물 중 수집 실패 및 데이터 수집이 완료되지 않은 항목을 확인하고 제거합니다. 

In [ ]:
# 수집 제대로 안된 데이터가 얼마나 있는지 확인하기
# --------------------------------------------------------------------------
# 1. 분석할 폴더 경로 - 3번 작업 결과
# --------------------------------------------------------------------------
TARGET_PATH = r'E:/데일리프로젝트/251120_카카오톡 네이버기사,블로그/collected_main_text_blog'

# --------------------------------------------------------------------------
# 2. 통계 변수 초기화
# --------------------------------------------------------------------------
total_blog_count = 0      # 1. 총 수집 블로그 개수
total_fail_count = 0      # 2. 페이지 요청 실패 개수 (1, 2단계 포함)
total_unknown_date = 0    # 3. 날짜 미상 개수

# --------------------------------------------------------------------------
# 3. 파일 순회 및 분석
# --------------------------------------------------------------------------
if not os.path.exists(TARGET_PATH):
    print(f"오류: 해당 폴더 경로가 존재하지 않습니다.\n{TARGET_PATH}")
else:
    pkl_files = [f for f in os.listdir(TARGET_PATH) if f.endswith('.pkl')]
    
    print(f"분석 시작: 총 {len(pkl_files)}개의 파일을 확인합니다.")
    print("-" * 60)

    for filename in pkl_files:
        file_path = os.path.join(TARGET_PATH, filename)
        
        try:
            df = pd.read_pickle(file_path)
            
            # (1) 현재 파일의 행 개수 더하기
            current_count = len(df)
            total_blog_count += current_count
            
            # (2) 페이지 요청 실패 개수 확인
            # '본문' 컬럼에 "페이지 요청 실패"라는 문자열이 포함된 행의 개수 - NaN 값은 제외하고 문자열로 변환 후 체크
            if '본문' in df.columns:
                fails = df[df['본문'].astype(str).str.contains("페이지 요청 실패", na=False)]
                current_fails = len(fails)
                total_fail_count += current_fails
            else:
                current_fails = 0

            # (3) 날짜 미상 개수 확인
            # '작성일' 컬럼이 "날짜 미상"인 행의 개수
            if '작성일' in df.columns:
                unknowns = df[df['작성일'] == "날짜 미상"]
                current_unknowns = len(unknowns)
                total_unknown_date += current_unknowns
            else:
                current_unknowns = 0
            
            print(f"[{filename}] 전체: {current_count} | 실패: {current_fails} | 날짜미상: {current_unknowns}")

        except Exception as e:
            print(f"[{filename}] 읽기 오류 발생: {e}")

    # --------------------------------------------------------------------------
    # 4. 최종 결과 출력
    # --------------------------------------------------------------------------
    print("-" * 60)
    print(f"[ 최종 분석 결과 ]")
    print(f"1. 총 수집 블로그 개수      : {total_blog_count:,} 개")
    print(f"2. 페이지 요청 실패 (1+2단계) : {total_fail_count:,} 개")
    print(f"3. 날짜 미상 개수           : {total_unknown_date:,} 개")
    
    # 성공률 계산 (총 개수 - 실패 개수) / 총 개수
    if total_blog_count > 0:
        success_rate = ((total_blog_count - total_fail_count) / total_blog_count) * 100
        print(f"-> 요청 성공률               : {success_rate:.2f}%")
    print("-" * 60)

In [ ]:
# 수집 제대로 안된 데이터 제거
# --------------------------------------------------------------------------
# 1. 경로 설정
# --------------------------------------------------------------------------
# 분석할 폴더 경로 - 3번 작업 결과
INPUT_PATH = r'E:/데일리프로젝트/251120_카카오톡 네이버기사,블로그/collected_main_text_blog'

# 정제된 데이터 저장용 폴더
OUTPUT_PATH = r"E:데일리프로젝트/251120_카카오톡 네이버기사,블로그/blog_cleaned_data"

# --------------------------------------------------------------------------
# 2. 폴더 생성 및 작업 시작
# --------------------------------------------------------------------------
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)
    print(f"결과 저장을 위한 폴더를 생성했습니다: {OUTPUT_PATH}")

pkl_files = [f for f in os.listdir(INPUT_PATH) if f.endswith('.pkl')]
print(f"총 {len(pkl_files)}개의 파일에 대해 정제 작업을 시작합니다.")
print("-" * 60)

total_removed = 0

for filename in pkl_files:
    source_path = os.path.join(INPUT_PATH, filename)
    save_path = os.path.join(OUTPUT_PATH, filename)
    
    try:
        df = pd.read_pickle(source_path)
        original_count = len(df)
        
        # ------------------------------------------------------------------
        # [정제 로직] 실패 및 날짜 미상 제거
        # ------------------------------------------------------------------
        
        # 1. '본문'에 '페이지 요청 실패'가 포함된 행 제거 - NaN 값 오류 방지를 위해 astype(str) 사용
        condition_fail = df['본문'].astype(str).str.contains("페이지 요청 실패")
        
        # 2. '작성일'이 '날짜 미상'인 행 제거 (!=)
        condition_unknown = (df['작성일'] == "날짜 미상")
        
        # 두 조건 중 하나라도 해당되면 제거 대상 -> 즉, 둘 다 해당하지 않는 것만 남김
        df_cleaned = df[~condition_fail & ~condition_unknown].copy()
        
        df_cleaned.reset_index(drop=True, inplace=True)      
      
        # 제거된 개수 계산
        removed_count = original_count - len(df_cleaned)
        total_removed += removed_count
        
        # 정제된 파일 저장
        df_cleaned.to_pickle(save_path)
        
        print(f"[{filename}] 완료 | 기존: {original_count} -> 정제후: {len(df_cleaned)} (삭제: {removed_count})")
        
    except Exception as e:
        print(f"[{filename}] 처리 중 오류 발생: {e}")

print("-" * 60)
print(f"모든 작업이 완료되었습니다.")
print(f"총 삭제된 행(실패+날짜미상): {total_removed} 개")
print(f"정제된 파일은 '{OUTPUT_PATH}' 폴더에 저장되었습니다.")

In [ ]:
# 정제 후 파일 재확인
# --------------------------------------------------------------------------
# 1. 분석할 폴더 경로 설정
# --------------------------------------------------------------------------
TARGET_PATH = r"E:데일리프로젝트/251120_카카오톡 네이버기사,블로그/blog_cleaned_data"

# --------------------------------------------------------------------------
# 2. 통계 변수 초기화
# --------------------------------------------------------------------------
total_blog_count = 0      # 1. 총 수집 블로그 개수
total_fail_count = 0      # 2. 페이지 요청 실패 개수 (1, 2단계 포함)
total_unknown_date = 0    # 3. 날짜 미상 개수

# --------------------------------------------------------------------------
# 3. 파일 순회 및 분석
# --------------------------------------------------------------------------
if not os.path.exists(TARGET_PATH):
    print(f"오류: 해당 폴더 경로가 존재하지 않습니다.\n{TARGET_PATH}")
else:
    pkl_files = [f for f in os.listdir(TARGET_PATH) if f.endswith('.pkl')]
    
    print(f"분석 시작: 총 {len(pkl_files)}개의 파일을 확인합니다.")
    print("-" * 60)

    for filename in pkl_files:
        file_path = os.path.join(TARGET_PATH, filename)
        
        try:
            df = pd.read_pickle(file_path)
            
            current_count = len(df)
            total_blog_count += current_count
            
            if '본문' in df.columns:
                fails = df[df['본문'].astype(str).str.contains("페이지 요청 실패", na=False)]
                current_fails = len(fails)
                total_fail_count += current_fails
            else:
                current_fails = 0

            if '작성일' in df.columns:
                unknowns = df[df['작성일'] == "날짜 미상"]
                current_unknowns = len(unknowns)
                total_unknown_date += current_unknowns
            else:
                current_unknowns = 0
            
            print(f"[{filename}] 전체: {current_count} | 실패: {current_fails} | 날짜미상: {current_unknowns}")

        except Exception as e:
            print(f"[{filename}] 읽기 오류 발생: {e}")

    # --------------------------------------------------------------------------
    # 4. 최종 결과 출력
    # --------------------------------------------------------------------------
    print("-" * 60)
    print(f"[ 최종 분석 결과 ]")
    print(f"1. 총 수집 블로그 개수      : {total_blog_count:,} 개")
    print(f"2. 페이지 요청 실패 (1+2단계) : {total_fail_count:,} 개")
    print(f"3. 날짜 미상 개수           : {total_unknown_date:,} 개")
    
    # 성공률 계산 (총 개수 - 실패 개수) / 총 개수
    if total_blog_count > 0:
        success_rate = ((total_blog_count - total_fail_count) / total_blog_count) * 100
        print(f"-> 요청 성공률               : {success_rate:.2f}%")
    print("-" * 60)

# --------------------------------------------------------------------------
# 5. 실제 출력해 볼 파일 경로 설정
# --------------------------------------------------------------------------
file_to_check = 'E:데일리프로젝트/251120_카카오톡 네이버기사,블로그/blog_cleaned_data/crawled_filtered_combined_1_카카오톡_업데이트.pkl'


# --------------------------------------------------------------------------
# 6. 파일 불러오기 및 기본 정보 확인
# --------------------------------------------------------------------------
try:
    df_check = pd.read_pickle(file_to_check)

    print("파일을 성공적으로 불러왔습니다!")
    print("-" * 30)

    print("[ 데이터프레임 정보 ]")
    df_check.info()
    print("-" * 30)

    print("[ 상위 5개 데이터 ]")
    display(df_check.head())
    print("-" * 30)

    # --------------------------------------------------------------------------
    # 7. 본문 내용 상세 확인
    # --------------------------------------------------------------------------
    print("[ 첫 번째 행의 본문 내용 전문 ]")
    print(df_check.loc[0, '본문'])
    
except FileNotFoundError:
    print(f"오류: 파일을 찾을 수 없습니다. 경로를 다시 확인해주세요.\n경로: {file_to_check}")
except Exception as e:
    print(f"파일을 읽는 중 오류가 발생했습니다: {e}")

### 5. 파일별 자연어 처리 작업 진행
- 목표 : 자연어 분석을 위해서 수집된 블로그 <b>'제목과 본문'</b>에서 특문, 이모티콘, 자음과 모음만 있는 경우를 제외하는 전처리 작업 진행합니다. 
- 핵심 : 숫자와 영문은 제거되지 않도록 처리가 필요합니다. 

In [ ]:
# 이모지 제거를 위해서 인스톨
!pip install emoji

import re
import emoji

# 자연어 분석을 위한 함수 만들기 & 테스트
def clean_text_with_lib(text):
    text = str(text)
    
    # 1. [수정됨] emoji 라이브러리를 사용하여 모든 이모티콘을 찾아 빈 문자열로 바꾸기
    text = emoji.replace_emoji(text, replace='')
    
    # 2. 한글, 영어, 숫자를 제외한 특수문자 제거
    text = re.sub(r'[^A-Za-z0-9가-힣ㄱ-ㅎㅏ-ㅣ\s]', '', text)
    
    # 3. 자음 또는 모음만 반복되는 경우 제거 (ㅋㅋㅋ, ㅜㅜㅜ)
    text = re.sub(r'([ㄱ-ㅎㅏ-ㅣ])\1{2,}', '', text)
    
    # 4. 여러 개의 공백을 하나의 공백으로 변경
    text = re.sub(r'\s+', ' ', text)
    
    # 5. 앞뒤 공백 제거
    text = text.strip()
    return text

# 테스트 코드 -> 결과 확인 후 반영
test_review1 = "업데이트 최악 1점 드립니다.ㅠㅠㅠ UI 돌려놔!!!! #카카오"
test_review2 = "ㅋㅋㅋ Only salvation is Jesus, believe in Jesus 🙏"
test_review3 = "이모티콘😄은 좋은데, 앱 속도가 너무 느려요..."
test_review4 = "0점이 없어서 1점남김ㅋㅋ"
print(f"원본: {test_review1} -> 정제 후: {clean_text_with_lib(test_review1)}")
print(f"원본: {test_review2} -> 정제 후: {clean_text_with_lib(test_review2)}")
print(f"원본: {test_review3} -> 정제 후: {clean_text_with_lib(test_review3)}")
print(f"원본: {test_review4} -> 정제 후: {clean_text_with_lib(test_review4)}")

In [ ]:
# 4번 결과로 획득한 .pkl 파일들이 있는 폴더 경로
INPUT_PATH = "E:/데일리프로젝트/251120_카카오톡 네이버기사,블로그/blog_cleaned_data"

# 정제 완료된 .pkl 파일을 저장할 폴더 경로 지정
OUTPUT_PATH = "E:/데일리프로젝트/251120_카카오톡 네이버기사,블로그/cleaned_main_text_blog"

# 텍스트 정제를 적용할 원본 컬럼명 - 본문 / 제목 모두 적용 필요
SOURCE_COLUMN = '본문'

# 정제된 텍스트를 저장할 새로운 컬럼명 - 본문_정제 / 제목_정제
TARGET_COLUMN = '본문_정제'

# 실 데이터에 반영하기
from tqdm.auto import tqdm

# 출력 폴더가 없으면 새로 생성
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)
    print(f"'{OUTPUT_PATH}' 폴더를 생성했습니다.")

# 입력 폴더에 있는 모든 파일 목록 가져오기
file_list = os.listdir(INPUT_PATH)

print("\n" + "="*50)
print("폴더 내 모든 .pkl 파일에 대한 텍스트 정제 작업을 시작합니다.")
print(f"대상 폴더: {INPUT_PATH}")
print("="*50 + "\n")

for filename in file_list:
    # .pkl 파일만 대상
    if filename.endswith('.pkl'):
        # 파일 경로 조합
        input_filepath = os.path.join(INPUT_PATH, filename)
        
        print(f"[{filename}] 파일 처리 중...")
        
        try:
            df = pd.read_pickle(input_filepath)
            
            if SOURCE_COLUMN not in df.columns:
                print(f" -> 경고: '{SOURCE_COLUMN}' 컬럼이 없어 건너뜁니다.")
                continue

            # 함수 적용 후 새로운 컬럼 생성
            df[TARGET_COLUMN] = df[SOURCE_COLUMN].apply(clean_text_with_lib)
            
            # 결과를 새로운 파일명으로 변경하여 pkl 파일로 저장
            output_filename = f"cleaned_{filename}"
            output_filepath = os.path.join(OUTPUT_PATH, output_filename)
            df.to_pickle(output_filepath)
            
            print(f" -> 완료! '{output_filename}'으로 저장되었습니다.\n")

        except Exception as e:
            print(f" -> 오류 발생: {filename} 처리 중 문제가 발생했습니다. 오류: {e}\n")

print("모든 텍스트 정제 작업이 성공적으로 완료되었습니다.")

In [ ]:
# 변경 후 파일 체크
# --------------------------------------------------------------------------
# 1. 파일 경로 설정
# --------------------------------------------------------------------------
file_to_check = 'E:/데일리프로젝트/251120_카카오톡 네이버기사,블로그/cleaned_main_text_blog/cleaned_crawled_filtered_combined_1_카카오톡_업데이트.pkl'


# --------------------------------------------------------------------------
# 2. 파일 불러오기 및 기본 정보 확인
# --------------------------------------------------------------------------
try:
    df_check = pd.read_pickle(file_to_check)

    print("파일을 성공적으로 불러왔습니다!")
    print("-" * 30)

    # 데이터의 전체적인 구조 확인 (행/열 개수, 컬럼별 타입 등)
    print("[ 데이터프레임 정보 ]")
    df_check.info()
    print("-" * 30)

    # 상위 5개 데이터 확인
    print("[ 상위 10개 데이터 ]")
    display(df_check.head(10))
    print("-" * 30)

    # --------------------------------------------------------------------------
    # 3. 본문 내용 상세 확인
    # --------------------------------------------------------------------------
    # 첫 번째 행의 '본문' 내용을 실제로 출력해보기
    print("[ 첫 번째 행의 본문 내용 전문 ]")
    print(df_check.loc[1, '본문'])
    
except FileNotFoundError:
    print(f"오류: 파일을 찾을 수 없습니다. 경로를 다시 확인해주세요.\n경로: {file_to_check}")
except Exception as e:
    print(f"파일을 읽는 중 오류가 발생했습니다: {e}")

### 6. 최종 파일 통합
- 목표 : 키워드별로 생성된 pkl 파일을 하나의 파일로 최종 통합합니다. 

In [ ]:
# --------------------------------------------------------------------------
# 1. 설정
# --------------------------------------------------------------------------
# 5번 결과 파일
CLEANED_DATA_PATH = "E:/데일리프로젝트/251120_카카오톡 네이버기사,블로그/cleaned_main_text_blog"

# --------------------------------------------------------------------------
# 2. 모든 파일을 하나로 합치기 (전체 통계를 위해)
# --------------------------------------------------------------------------
# 폴더 내의 모든 정제된 pkl 파일 경로를 리스트 만들기
all_cleaned_files = [os.path.join(CLEANED_DATA_PATH, f) for f in os.listdir(CLEANED_DATA_PATH) if f.endswith('.pkl')]

# 모든 데이터프레임을 읽어와 하나의 리스트로 병합
df_list = [pd.read_pickle(file) for file in all_cleaned_files]

combined_df = pd.concat(df_list, ignore_index=True)

print("모든 정제된 블로그 파일 통합 완료!")
print(f" -> 총 블로그글 수: {len(combined_df)} 개")
print("-" * 30)

combined_df.to_pickle("final_blog_all_articles.pkl")
print("\n블로그 파일 통합 데이터를 'final_blog_all_articles.pkl'로 저장했습니다.")